In [43]:
#read data(exclude replies) 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import SnowballStemmer
import pandas as pd

text = pd.read_csv('Jan6(excludes replies).csv')
print('The number of tweets (excludes replies):',len(text))
#Text cleaning
import gensim
import gensim.corpora as corpora
import re
stemmer = SnowballStemmer('english')
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words += ['cultured','clean','cultivated','meat']
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['tweets']).split():
        if '@' not in words: #remove @users
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = words.lower()# lower form
                        words = re.sub(r'[^a-zA-Z]', ' ', words) #replace non-alphabets characters with space. From "can't" to "can t"
                        if len(words)>3:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if len(token) >1:
            if token not in stop_words:
                token = porter.stem(token) #root form
                doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]




import guidedlda
import numpy as np
import gensim
topic_tweets=[]
other_tweets=[]

keywords_list = ['environ','environment','carbon','climat','greenhous','emiss', #Environmental Impact
                   'plant','vegan','vegetarian','animal','without','slaughter',
                   'cruelti','welfar','cell','cellular','anim','kill',# Animal Welfare
                  'antibiot','antimicrobi','resist','safeti','contamin','consumpt','popul',
                  'demand','secur','wast','industri','suppli','convent','regular','system','market','consum','store','groceri', #Food System
                  'seafood','sea','ocean','fish','aquacultur','shrimp',#Seafood
                  'delici','tast','tender','textur','juici','test','chicken','duck','poultri','egg',#Poultry
                  'seri','fund','funder','rais','invest','investor','dollar','pound','announc', 
                  'happi','thrill','excit', #Announcement
                  'thank','thankyou','support','shoutout','help','shout', #Appreciation
                  'confer','regist','live','symposium','stream','livestream','ceo','speak','summit', #conference and summit
                  'check','post','paper','interview','articl','blog','news','break','episod','discuss','vote','stori' #Media
                  'hire','join','team','career']#Hiring Information
for index,i in text.iterrows():
    count = 0 
    for word in i['processed'].split():
        if word in keywords_list:
            count+=1 #if any keyword occur in a tweet, the tweet may belong to one of the above themes
    if count !=0: #if none of the keywords occur in a tweet, the tweet doesnot belong to any themes we found
        topic_tweets.append(i['processed'])
    else:
        other_tweets.append(i['processed'])
print('The number of tweets within 10 topics:',len(topic_tweets))
print('The number of other tweets:',len(other_tweets))


seed_topic_list = [['environ','environment','carbon','climat','greenhous','emiss'], #Environmental Impact
                   ['plant','vegan','vegetarian','animal','without','slaughter',
                   'cruelti','welfar','cell','cellular','anim','kill'],# Animal Welfare
                  ['antibiot','antimicrobi','resist','safeti','contamin','consumpt','popul','demand','secur','wast',
                   'industri','suppli','convent','regular','system','market','consum','store','groceri'], #Food System
                  ['seafood','sea','ocean','fish','aquacultur','shrimp'],#Seafood
                  ['delici','tast','tender','textur','juici','test','chicken','duck','poultri','egg'],#Poultry
                  ['seri','fund','funder','rais','invest','investor','dollar','pound','announc', 
                  'happi','thrill','excit','honor',], #Announcement
                  ['thank','thankyou','support','shoutout','help','shout'], #Appreciation
                  ['confer','regist','live','symposium','stream','livestream','speak','summit'], #conference and summit
                  ['check','post','paper','interview','articl','blog','news','break','episod','discuss','vote','stori'], #Media
                  ['hire','join','team','career']]#Hiring Information


from sklearn.feature_extraction.text import CountVectorizer
model = guidedlda.GuidedLDA(n_topics=10,n_iter=200,random_state=1,refresh=10,alpha=0.1,eta=0.01)
vectorizer = CountVectorizer(min_df = 2)
X = vectorizer.fit_transform(topic_tweets)

vocab = vectorizer.get_feature_names()
word2id = dict((v,idx) for idx,v in enumerate(vocab))
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

model.fit(X.toarray(),seed_topics=seed_topics,seed_confidence=0.35)
topic_word = model.topic_word_
n_top_words = 40
vocab = tuple(vocab)

for i, topic_dist in enumerate(topic_word):
    print('\n')
    print('Topic:',i)
    words_probability = np.array(-topic_dist)
    for index in range(n_top_words):
        print(round(abs(np.sort(words_probability))[:(n_top_words)][index],4),'*',
              np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1][index],sep='',end='  ')

X = X.toarray()
doc_topic = model.transform(X)
topic_number = []
number = []
for i in range(len(doc_topic)):
    topic_number.append(doc_topic[i].argmax())#得到每篇tweets對應概率最高的topic
    number.append('1')
data = pd.DataFrame({'Company':[i for i in topic_tweets_Company],
                    'number':[i for i in number],
                     'topic_number':[i for i in topic_number]})
print(data.groupby(['Company','topic_number'])['number'].count())


The number of tweets (excludes replies): 6501


INFO:guidedlda:n_documents: 4275
INFO:guidedlda:vocab_size: 2911
INFO:guidedlda:n_words: 46965
INFO:guidedlda:n_topics: 10
INFO:guidedlda:n_iter: 200


The number of tweets within 10 topics: 4275
The number of other tweets: 2226


INFO:guidedlda:<0> log likelihood: -520717
INFO:guidedlda:<10> log likelihood: -371787
INFO:guidedlda:<20> log likelihood: -359300
INFO:guidedlda:<30> log likelihood: -355065
INFO:guidedlda:<40> log likelihood: -352176
INFO:guidedlda:<50> log likelihood: -351119
INFO:guidedlda:<60> log likelihood: -350436
INFO:guidedlda:<70> log likelihood: -349057
INFO:guidedlda:<80> log likelihood: -348911
INFO:guidedlda:<90> log likelihood: -348454
INFO:guidedlda:<100> log likelihood: -348266
INFO:guidedlda:<110> log likelihood: -347980
INFO:guidedlda:<120> log likelihood: -347744
INFO:guidedlda:<130> log likelihood: -347479
INFO:guidedlda:<140> log likelihood: -347126
INFO:guidedlda:<150> log likelihood: -346729
INFO:guidedlda:<160> log likelihood: -346768
INFO:guidedlda:<170> log likelihood: -347136
INFO:guidedlda:<180> log likelihood: -346490
INFO:guidedlda:<190> log likelihood: -346690
INFO:guidedlda:<199> log likelihood: -346682




Topic: 0
0.0418*food  0.0304*chang  0.0224*climat  0.0204*system  0.018*sustain  0.0171*world  0.0169*anim  0.0145*global  0.0135*health  0.0124*help  0.0116*impact  0.0112*human  0.011*industri  0.0108*suppli  0.0104*environment  0.0098*product  0.0096*feed  0.0092*need  0.0084*pandem  0.0082*agricultur  0.008*solut  0.008*time  0.0078*chain  0.0078*challeng  0.0073*grow  0.0073*reduc  0.0071*potenti  0.0065*creat  0.0063*earth  0.0061*today  0.0059*could  0.0059*mission  0.0059*issu  0.0057*demand  0.0055*environ  0.0055*news  0.0055*live  0.0055*way  0.0053*consum  0.0053*good  

Topic: 1
0.0702*anim  0.0528*cell  0.0304*without  0.0282*produc  0.0219*grow  0.0214*real  0.0201*base  0.0166*slaughter  0.0164*make  0.0153*free  0.0136*startup  0.0131*use  0.0126*need  0.0118*product  0.0116*chicken  0.0116*develop  0.0106*cow  0.0106*protein  0.0103*farm  0.0103*plant  0.0096*grown  0.0096*rais  0.0091*compani  0.0078*learn  0.0075*cultur  0.0073*like  0.0073*whole  0.0073*cost  0.0